# Semgrex를 활용한 인과추출
---

# 1. Semgrex Example(인과추출은X)

## stanfordNLP 예시

In [26]:
from stanfordnlp.server import CoreNLPClient

# example text
print('---')
print('input text')
print('')

text = "Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    # Use tokensregex patterns to find who wrote a sentence.
    pattern = '([ner: PERSON]+) /wrote/ /an?/ []{0,3} /sentence|article/'
    matches = client.tokensregex(text, pattern)
    # sentences contains a list with matches for each sentence.
    
    assert len(matches["sentences"]) == 3
    # length tells you whether or not there are any matches in this
#"length"는 여기에 일치하는 항목이 있는지 여부를 알려줌
    assert matches["sentences"][1]["length"] == 1
    # You can access matches like most regex groups.
    matches["sentences"][1]["0"]["text"] == "Chris wrote a simple sentence"
    matches["sentences"][1]["0"]["1"]["text"] == "Chris"

    # Use semgrex patterns to directly find who wrote what.
    pattern = '{word:wrote} >nsubj {}=subject >dobj {}=object'
    matches = client.semgrex(text, pattern)
    
    print(matches["sentences"])
    print(len(matches["sentences"]))
    print(matches["sentences"][1]["length"])    

    # sentences contains a list with matches for each sentence.
    assert len(matches["sentences"]) == 3
    # length tells you whether or not there are any matches in this
    assert matches["sentences"][1]["length"] == 1
    # You can access matches like most regex groups.
    matches["sentences"][1]["0"]["text"] == "wrote"
    matches["sentences"][1]["0"]["$subject"]["text"] == "Chris"
    matches["sentences"][1]["0"]["$object"]["text"] == "sentence"

---
input text

Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people.
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-765bfddbfc584c05.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
[{'length': 0}, {'0': {'text': 'wrote', 'begin': 1, 'end': 2, '$subject': {'text': 'Chris', 'begin': 0, 'end': 1}, '$object': {'text': 'sentence', 'begin': 4, 'end': 5}}, 'length': 1}, {'length': 0}]
3
1


## Stanza 예시

In [6]:
from stanfordnlp.server import CoreNLPClient

In [17]:
text = "Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."
with CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse'],
        timeout=30000,
        memory='16G') as client:

    # Use semgrex patterns to directly find who wrote what.
    pattern = '{word:wrote} >nsubj {}=subject >obj {}=object'
    matches = client.semgrex(text, pattern)
    print("----------------------------------------------------------------------")
    print(matches)
#     # sentences contains a list with matches for each sentence.
#     print(len(matches["sentences"])) # prints: 3
#     # length tells you whether or not there are any matches in this
#     print(matches["sentences"][1]["length"]) # prints: 1
#     # You can access matches like most regex groups.
#     print(matches["sentences"][1]["0"]["text"]) # prints: "wrote"
#     print(matches["sentences"][1]["0"]["$subject"]["text"]) # prints: "Chris"
#     print(matches["sentences"][1]["0"]["$object"]["text"]) # prints: "sentence"

#     # Tregex example
#     pattern = 'NP'
#     matches = client.tregex(text, pattern)
#     # You can access matches similarly
#     print(matches['sentences'][1]['1']['match']) # prints: "(NP (DT a) (JJ simple) (NN sentence))\n"

Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-6aacae97583446a3.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse
----------------------------------------------------------------------
{'sentences': [{'length': 0}, {'length': 0}, {'length': 0}]}


## test

In [1]:
from stanfordnlp.server import CoreNLPClient
# example text
print('---')
print('input text')
print('')
text = "Over thinking can increase anxiety and cause insomnia."
print(text)
# set up the client
print('---')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner','depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
# sentence = ann.sentence[0] #문장 3개 중 (첫)번째 문장
#print(sentence.basicDependencies) #의존성구문 분석
#print(sentence.token[5]) #첫 문장 안의 (첫)단어를 끊어 분석(token)
#print(sentence.token[0].pos) #한 토큰 분석한 결과 중 pos만 따로 확인
#---
#print(sentence.ner) #주어 추출
#print(sentence.mentions[0]) # 주어 추출2?
#print(ann.corefChain) #뭔지모르겠음

#pattern = '([ner: PERSON]+) /wrote/ /an?/ []{0,3} /sentence|article/'
#matches = client.tokensregex(text, pattern)
#assert len(matches["sentences"]) == 3
#print(matches)
#assert matches["sentences"][1]["length"] == 1
#matches["sentences"][1]["0"]["text"] == "Chris wrote a simple sentence"
#matches["sentences"][1]["0"]["1"]["text"] == "Chris"

    pattern = '{} = subj < csubj ({word:/cause/} = target >dobj {} = cause)'
    matches = client.semgrex(text, pattern)
    #print(matches["sentences"][0]["0"])
    print("------------------------")
    print(matches["sentences"])
    

---
input text

Over thinking can increase anxiety and cause insomnia.
---
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-58eed0f0e0384693.props -preload tokenize,ssplit,pos,lemma,ner,depparse,coref
------------------------
[{'0': {'text': 'thinking', 'begin': 1, 'end': 2, '$cause': {'text': 'insomnia', 'begin': 7, 'end': 8}, '$subj': {'text': 'thinking', 'begin': 1, 'end': 2}, '$target': {'text': 'cause', 'begin': 6, 'end': 7}}, 'length': 1}]


In [15]:
from stanfordnlp.server import CoreNLPClient
# example text
print('---')
print('input text')
print('')
text = "Over thinking can increase anxiety and cause insomnia."
print(text)
# set up the client
print('---')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner','depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
# sentence = ann.sentence[0] #문장 3개 중 (첫)번째 문장
#print(sentence.basicDependencies) #의존성구문 분석
#print(sentence.token[5]) #첫 문장 안의 (첫)단어를 끊어 분석(token)
#print(sentence.token[0].pos) #한 토큰 분석한 결과 중 pos만 따로 확인
#---
#print(sentence.ner) #주어 추출
#print(sentence.mentions[0]) # 주어 추출2?
#print(ann.corefChain) #뭔지모르겠음

#pattern = '([ner: PERSON]+) /wrote/ /an?/ []{0,3} /sentence|article/'
#matches = client.tokensregex(text, pattern)
#assert len(matches["sentences"]) == 3
#print(matches)
#assert matches["sentences"][1]["length"] == 1
#matches["sentences"][1]["0"]["text"] == "Chris wrote a simple sentence"
#matches["sentences"][1]["0"]["1"]["text"] == "Chris"

    pattern = '{} = subj < csubj ({word:/cause/} = target >dobj {} = cause)'
    matches = client.semgrex(text, pattern)
    #print(matches["sentences"][0]["0"])
    print("------------------------")
    print(matches["sentences"])
    

---
input text

Over thinking can increase anxiety and cause insomnia.
---
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c58e0524599347b0.props -preload tokenize,ssplit,pos,lemma,ner,depparse,coref
------------------------
[{'0': {'text': 'thinking', 'begin': 1, 'end': 2, '$cause': {'text': 'insomnia', 'begin': 7, 'end': 8}, '$subj': {'text': 'thinking', 'begin': 1, 'end': 2}, '$target': {'text': 'cause', 'begin': 6, 'end': 7}}, 'length': 1}]


---

# Semgrex 인과추출 Rule

## Rule1

In [1]:
from stanfordnlp.server import CoreNLPClient

# text = "Stress causes insomnia."
# text = "Missing someone causes insomnia."  #Missing someone으로 추출이 안되네?
# text = "Money only causes stress and conflict."
# text = "Cell phone radiation can cause insomnia"
# text = "My neck just made my headache 100x worse."  # >dobj로 하면 추출 안됨 패턴 바꿔야함
# text = "stress doesn't cause depression"
text = "jones et al demonstrated that the presence of sarcopenia in copd affect the response to pr and in some patients pr might lead to a resolution of sarcopenia"


print(text)
# set up the client
print('---')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner','depparse','coref'], timeout=150000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    
#     lemma:/(cause|stimulate|make|derive|trigger|result|lead)/
#     pattern = '{}=cause <nsubj ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >dobj {}=effect)'
    pattern = "{}=cause <nsubj ({lemma:/(cause|stimulate|make|derive|trigger|result|lead|incur|induce|decrease|increase|drive|stimulate|reduce|influence|effect|derive|affect)/}=trigger >dobj {}=effect!>neg{pos:RB})"
    
    matches = client.semgrex(text, pattern)
    print("------------------------")
    print(matches)
    print(matches["sentences"] == [{'length': 0}])
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])

jones et al demonstrated that the presence of sarcopenia in copd affect the response to pr and in some patients pr might lead to a resolution of sarcopenia
---
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 150000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7bcac9226c9b464b.props -preload tokenize,ssplit,pos,lemma,ner,depparse,coref
------------------------
{'sentences': [{'0': {'text': 'presence', 'begin': 6, 'end': 7, '$effect': {'text': 'response', 'begin': 13, 'end': 14}, '$cause': {'text': 'presence', 'begin': 6, 'end': 7}, '$trigger': {'text': 'affect', 'begin': 11, 'end': 12}}, 'length': 1}]}
False
------------------------
presence
affect
response


## Rule2

In [125]:
from stanfordnlp.server import CoreNLPClient

# text = "Over thinking can increase anxiety and cause insomnia."
text = "Missing someone causes insomnia"
print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    ann = client.annotate(text)
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    pattern = '{}=cause <csubj ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >dobj {}=effect)'
    matches = client.semgrex(text, pattern)

    print(matches["sentences"])
#     print(len(matches["sentences"]))
#     print(matches["sentences"][0]["length"])

    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

Missing someone causes insomnia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-5a22e5366c3641d5.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
[{'0': {'text': 'Missing', 'begin': 0, 'end': 1, '$cause': {'text': 'insomnia', 'begin': 3, 'end': 4}, '$subj': {'text': 'Missing', 'begin': 0, 'end': 1}, '$target': {'text': 'causes', 'begin': 2, 'end': 3}}, 'length': 1}]
insomnia
Missing
causes


## Rule3

In [14]:
from stanfordnlp.server import CoreNLPClient

# text = "My insomnia was caused by stress."
# text = "Stress was caused by insomnia"
text = "3 another study,4 followed copd patients for 1 year, observed a high rate of exacerbations in one subgroup – 37% of these were caused by bacteria, 10% by viruses, 12% by bacteria and viruses, while 14% were of undefined origin"


print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    
    # Use semgrex patterns to directly find who wrote what.
    pattern = '{}=effect <nsubjpass ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >/nmod:agent/ {}=cause)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    

'3 nevertheless, despite the availability of guideline recommendations,4–6 diagnostic confusion between copd and asthma appears common, and often it is very difficult in a patient with airway obstruction to decide whether the obstruction is caused by asthma or copd
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-8145bc65f7d84b91.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'obstruction', 'begin': 38, 'end': 39, '$effect': {'text': 'obstruction', 'begin': 38, 'end': 39}, '$cause': {'text': 'asthma', 'begin': 42, 'end': 43}, '$trigger': {'text': 'caused', 'begin': 40, 'end': 41}}, '1': {'text': 'obstruction',

## Rule4.1

In [6]:
from stanfordnlp.server import CoreNLPClient

text = "Stress is a reason of my insomnia"
# text = "School is the main cause of my stress."
# text = "You're the cause of my headaches."


print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    
    pattern = "{}=cause <nsubj ({lemma:/(reason|cause|trigger)/}=trigger >/nmod:of/ {}=effect !>neg {pos:RB})"
#     pattern = '{}=cause <nsubj ({lemma:/(reason|cause)/}=trigger >/nmod:of/ {}=effect)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

Stress is a reason of my insomnia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-38f72783901847e2.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'Stress', 'begin': 0, 'end': 1, '$effect': {'text': 'insomnia', 'begin': 6, 'end': 7}, '$cause': {'text': 'Stress', 'begin': 0, 'end': 1}, '$trigger': {'text': 'reason', 'begin': 3, 'end': 4}}, 'length': 1}]}
------------------------
Stress
reason
insomnia


## Rule4.2

In [4]:
from stanfordnlp.server import CoreNLPClient

text = "Insomnia is a result of stress"
# text = "School is the main cause of my stress."
# text = "You're the cause of my headaches."


print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    # Use semgrex patterns to directly find who wrote what.
    pattern = '{}=effect <nsubj ({lemma:/(result|trigger)/}=trigger >/nmod:of/ {}=cause !>neg {pos:RB})'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

Insomnia is a result of stress
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-aaadb506e4ab45fa.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'Insomnia', 'begin': 0, 'end': 1, '$effect': {'text': 'Insomnia', 'begin': 0, 'end': 1}, '$cause': {'text': 'stress', 'begin': 5, 'end': 6}, '$trigger': {'text': 'result', 'begin': 3, 'end': 4}}, 'length': 1}]}
------------------------
stress
result
Insomnia


## Rule5

In [117]:
from stanfordnlp.server import CoreNLPClient

text = "Insomnia was caused by overthinking"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    
    # Use semgrex patterns to directly find who wrote what.
    pattern = '{}=effect <nsubjpass ({lemma:/(result|reason|cause)/}=trigger >/advcl:by/ {}=cause)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])

Insomnia was caused by overthinking
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-294eb5c3883640c4.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'Insomnia', 'begin': 0, 'end': 1, '$pcomp': {'text': 'overthinking', 'begin': 4, 'end': 5}, '$nsubj': {'text': 'Insomnia', 'begin': 0, 'end': 1}, '$target': {'text': 'caused', 'begin': 2, 'end': 3}}, 'length': 1}]}
------------------------
Insomnia
caused
overthinking


## Rule6

In [29]:
from stanfordnlp.server import CoreNLPClient

# text = "Stress results to insomnia."
# text = "Night before first day of school always results in insomnia."
# text = "Nervous Stressed Leads to swollen eye & headaches."   #추출 안됌
# text = "Too many tears leads to headaches and heavy hearts."
# text = "stress lead to depression"
# text = "stress give rise to depression"
# text = 'stress gave rise to depression'
# text = 'stress led to depression'
# text = "stress lead to increase insomnia"



# A Cochrane review65 suggested that a short course, 3 to 7days, of systemic corticosteroids does not lead to increase in treatment failure or risk of relapse of AECOPD

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
#     pattern = '{}=cause </(nsubj|csubj|subj|compound)/ ({lemma:/(cause|stimulate|make|derive|trigger|result|lead|give)/}=trigger >/nmod:(to|in)/ {}=effect)'
    pattern = '{}=cause </(nsubj|csubj|subj|compound)/ ({lemma:/(cause|stimulate|make|derive|trigger|result|lead|incur|induce|decrease|increase|drive|stimulate|reduce|influence|effect|derive)/}=trigger >/nmod:(to|in)/ !{pos:CD}=effect !>neg {pos:RB})'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

jones et al demonstrated that the presence of sarcopenia in copd affect the response to pr and in some patients pr might lead to a resolution of sarcopenia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-1fb2467735424a48.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'pr', 'begin': 20, 'end': 21, '$effect': {'text': 'patients', 'begin': 19, 'end': 20}, '$cause': {'text': 'pr', 'begin': 20, 'end': 21}, '$trigger': {'text': 'lead', 'begin': 22, 'end': 23}}, '1': {'text': 'pr', 'begin': 20, 'end': 21, '$effect': {'text': 'resolution', 'begin': 25, 'end': 26}, '$cause': {'text': 'pr', 'begin': 20, 'end': 21}, '$t

## Rule7

In [88]:
from stanfordnlp.server import CoreNLPClient

# text= "depression resulted from stress"
text = 'depression results from stress'

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    pattern = '{}=effect </(nsubj|csubj|subj|compound)/ ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >/nmod:from/ {}=cause)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

depression results from stress
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-fbca2de3bb034cdd.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'depression', 'begin': 0, 'end': 1, '$cause': {'text': 'depression', 'begin': 0, 'end': 1}, '$causal': {'text': 'stress', 'begin': 3, 'end': 4}, '$target': {'text': 'results', 'begin': 1, 'end': 2}}, 'length': 1}]}
------------------------
stress
results
depression


## Rule8

In [21]:
from stanfordnlp.server import CoreNLPClient

# text= "stress leading to depression"
# text = "this in turn promotes airway and systemic infiammation leading to copd progression"


print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    pattern = '{}=cause >acl ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >/nmod:(to|in)/ {}=effect)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

loss of sirt1 correlates with tissue inhibitor of metalloproteinase -1 lysine acetylation and subsequent degradation of timp-1, the major anti-mmp protease, resulting in increased mmp-9 in human copd lung tissue
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7b131156dc7e4a84.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'inhibitor', 'begin': 6, 'end': 7, '$effect': {'text': 'mmp-9', 'begin': 26, 'end': 27}, '$cause': {'text': 'inhibitor', 'begin': 6, 'end': 7}, '$trigger': {'text': 'resulting', 'begin': 23, 'end': 24}}, 'length': 1}]}
------------------------
inhibitor
resulting
mmp-9


## Rule9

In [19]:
from stanfordnlp.server import CoreNLPClient

text= "stress giving rise to depression"


print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    pattern = '{}=cause >acl ({lemma:/give/}=trigger >/nmod:(to|in)/ {}=effect)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

systemic lead to treatment failure
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-f9de9de2d3de4e35.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'length': 0}]}
------------------------


KeyError: '0'

## Rule10(new)

In [9]:
from stanfordnlp.server import CoreNLPClient

#the reason(s) for/of B BE

# text= "the reason for insomnia is stress"
text= "the reason of insomnia is stress"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    pattern = "{}=cause !>neg{pos:RB} >cop {lemma:/(be)/}=be >nsubj ({lemma:/(reason)/}=trigger >/nmod:(for|of)/ {}=effect)"
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
#     print(matches["sentences"][0]["0"]["$be"]["text"])
#     print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

the reason of insomnia is stress
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-bcb76ce3fd304ae5.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'stress', 'begin': 5, 'end': 6, '$be': {'text': 'is', 'begin': 4, 'end': 5}, '$effect': {'text': 'insomnia', 'begin': 3, 'end': 4}, '$cause': {'text': 'stress', 'begin': 5, 'end': 6}, '$trigger': {'text': 'reason', 'begin': 1, 'end': 2}}, 'length': 1}]}
------------------------
stress
insomnia


## Rule11(new)

In [12]:
from stanfordnlp.server import CoreNLPClient

#DET effect of A BE B

text= "the result of stress is insomnia"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')
    
    pattern = '{}=effect !>neg{pos:RB} >cop {lemma:/(be)/}=be >nsubj ({lemma:/(result)/}=trigger >/nmod:(of)/ {}=cause)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$be"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

the result of stress is insomnia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-3ff8bd77cd264dd7.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'insomnia', 'begin': 5, 'end': 6, '$be': {'text': 'is', 'begin': 4, 'end': 5}, '$effect': {'text': 'insomnia', 'begin': 5, 'end': 6}, '$cause': {'text': 'stress', 'begin': 3, 'end': 4}, '$trigger': {'text': 'result', 'begin': 1, 'end': 2}}, 'length': 1}]}
------------------------
stress
is
result
insomnia


## Rule12(new)

In [18]:
from stanfordnlp.server import CoreNLPClient

#A BE DET reason(s) for B
text= "Stress is a reason for insomnia"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    pattern = '{lemma:/reason/}=trigger >nsubj {}=cause >/nmod:(of|for)/ {}=effect >cop {lemma:/be/}=be !>neg{pos:RB}'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$be"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

Stress is a reason for insomnia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-82c4bb9f106e47b4.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'reason', 'begin': 3, 'end': 4, '$be': {'text': 'is', 'begin': 1, 'end': 2}, '$effect': {'text': 'insomnia', 'begin': 5, 'end': 6}, '$cause': {'text': 'Stress', 'begin': 0, 'end': 1}, '$trigger': {'text': 'reason', 'begin': 3, 'end': 4}}, 'length': 1}]}
------------------------
Stress
is
reason
insomnia


## Rule13(new)

In [3]:
from stanfordnlp.server import CoreNLPClient

#A brings on B
text= "stress brings on insomnia"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    pattern = '{lemma:/bring/}=trigger >nsubj {}=cause >/(nmod:on|dobj)/{}=effect !>neg{pos:RB}'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
#     print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

stress brings on insomnia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-01a56ff9e1ff4039.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'brings', 'begin': 1, 'end': 2, '$effect': {'text': 'insomnia', 'begin': 3, 'end': 4}, '$cause': {'text': 'stress', 'begin': 0, 'end': 1}, '$trigger': {'text': 'brings', 'begin': 1, 'end': 2}}, 'length': 1}]}
------------------------
stress
insomnia


## Rule14(진행중. Rule5랑 비슷한데 왜 안되는지 모르겠음)

In [10]:
from stanfordnlp.server import CoreNLPClient

#A brings on B
text= "stress caused by insomnia"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    pattern = '{}=effect <nsubj ({lemma:/(cause)/}=trigger >/advcl:by/ {}=cause)'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    

stress caused by insomnia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-993c24b2234f439f.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'length': 0}]}
------------------------


KeyError: '0'

## Rule test

In [10]:
from stanfordnlp.server import CoreNLPClient

#A brings on B
text= "jones et al demonstrated that the presence of sarcopenia in copd does affect the response to pr and in some patients pr might lead to a resolution of sarcopenia"

print(text)

# set up the client
print('---------------------------------------------')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    # get the first sentence
    sentence = ann.sentence[0]
    print('---------------------------------------------')

    pattern = '{lemma:/(cause|stimulate|make|derive|trigger|result|lead|incur|induce|decrease|increase|drive|stimulate|reduce|influence|effect|derive|affect)/}=trigger >>/nmod:of/{}=cause >dobj{}=effect !>neg{pos:RB}'
    matches = client.semgrex(text, pattern)
    
    print(matches)
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
#     print(matches["sentences"][0]["0"]["$be"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    

jones et al demonstrated that the presence of sarcopenia in copd does affect the response to pr and in some patients pr might lead to a resolution of sarcopenia
---------------------------------------------
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-db2805f65a594ef7.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
---------------------------------------------
{'sentences': [{'0': {'text': 'affect', 'begin': 12, 'end': 13, '$effect': {'text': 'response', 'begin': 14, 'end': 15}, '$cause': {'text': 'sarcopenia', 'begin': 28, 'end': 29}, '$trigger': {'text': 'affect', 'begin': 12, 'end': 13}}, '1': {'text': 'affect', 'begin': 12, 'end': 13, '$effect': {'text': 'response', 'begin': 14, 'end': 15}, '$cause': {'text': 'sarcopenia',

## Rule(neg)

In [17]:
from stanfordnlp.server import CoreNLPClient

# text = "Stress causes insomnia."
# text = "Missing someone causes insomnia."  #Missing someone으로 추출이 안되네?
# text = "Money only causes stress and conflict."
# text = "Cell phone radiation can cause insomnia"
# text = "My neck just made my headache 100x worse."  # >dobj로 하면 추출 안됨 패턴 바꿔야함
text = "stress doesn't cause depression" 

# {pos:/VB.*/}
# {pos:/NN.*/}

print(text)
# set up the client
print('---')
print('starting up Java Stanford CoreNLP Server...')
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner','depparse','coref'], timeout=150000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)
    
#     lemma:/(cause|stimulate|make|derive|trigger|result|lead)/
    pattern = '{}=cause <nsubj ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >dobj {}=effect >neg {pos:/RB.*/}=not)'
    matches = client.semgrex(text, pattern)
    print("------------------------")
    print(matches)
    print(matches["sentences"] == [{'length': 0}])
    print("------------------------")
    print(matches["sentences"][0]["0"]["$cause"]["text"])
    print(matches["sentences"][0]["0"]["$trigger"]["text"])
    print(matches["sentences"][0]["0"]["$effect"]["text"])
    print(matches["sentences"][0]["0"]["$not"]["text"])

stress doesn't cause depression
---
starting up Java Stanford CoreNLP Server...
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 150000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-3afbe00a2ede4faa.props -preload tokenize,ssplit,pos,lemma,ner,depparse,coref
------------------------
{'sentences': [{'0': {'text': 'stress', 'begin': 0, 'end': 1, '$not': {'text': "n't", 'begin': 2, 'end': 3}, '$effect': {'text': 'depression', 'begin': 4, 'end': 5}, '$cause': {'text': 'stress', 'begin': 0, 'end': 1}, '$trigger': {'text': 'cause', 'begin': 3, 'end': 4}}, 'length': 1}]}
False
------------------------
stress
cause
depression
n't


# Causality Extraction(cause, effect)

## Causal Extraction

In [3]:
txt = open("pattern_test.txt", "r")
slist2=txt.read()
slist2 = slist2.replace("(",".")
slist2 = slist2.replace(")",".")
slist2 = slist2.replace("[",".")
slist2 = slist2.replace("]",".")
slist2 = slist2.replace(";",".")
slist2 = slist2.replace(":",".")
slist2 = slist2.replace(",",".")
# 쉼표(,)를 split안해주려고 했는데 이거 안하면 문장이 엄청 길어져서 패턴6에 걸치게됨
slist2 = slist2.split(".")
slist2 = list(filter(None, slist2))

def func1(n):
    if n == " ":
        return False
    return True

slist2 = list(filter(func1, slist2))
dlist=[]
for s in slist2:
#     if ("Depression" or "depression") in s:
    if ("insomnia" or "stress" or "headache") in s:
        dlist.append(s)

print(dlist)

[' insomnia', ' “Stress caused insomnia”', ' “Stress results in insomnia”', ' “Stress was caused by insomnia”', ' such as “Stress caused my insomnia”', ' where “Stress” is matched with the pattern “{} = subj” and “insomnia” is matched with the pattern “{} = cause', ' insomnia', ' Stress causes insomnia A ', ' Examples Over thinking can increase anxiety and cause insomnia', ' My insomnia was caused by stress', ' Stress is a reason of my insomnia {} = nsubj< nsubjpass ', ' Stress results to insomnia', ' 72 out of 3827 for insomnia ', ' The final causalities extracted were 41 for insomnia', ' similar phrases “missing someone causes insomnia”', ' “missing someone often causes insomnia”', ' and “missing someone causes insomnia like symptoms” were found', ' “Cell phone radiation can cause insomnia”', ' It also indicates that finding causal relationships for “headache” is more difficult than “insomnia” and “stress”', '27% Total # extracted causalities Qualitative analysis We further manually 

In [4]:
len(dlist)

27

In [9]:
from stanfordnlp.server import CoreNLPClient

# slist = ["Stress causes insomnia.", "Missing someone causes insomnia.", "Money only causes stress and conflict.", "My neck just made my headache 100x worse.", "Over thinking can increase anxiety and cause insomnia."]
plist = ['{}=cause <nsubj ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >dobj {}=effect)', '{}=cause <csubj ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >dobj {}=effect)', '{}=effect <nsubjpass ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >/nmod:agent/ {}=cause)', '{}=cause <nsubj ({lemma:/(result|reason|cause)/}=trigger >/nmod:of/ {}=effect)', '{}=effect <nsubjpass ({lemma:/(result|reason|cause)/}=trigger >/advcl:by/ {}=cause)', '{}=cause </(nsubj|csubj|subj|compound)/ ({lemma:/(cause|stimulate|make|derive|trigger|result|lead|give)/}=trigger >/nmod:(to|in)/ {}=effect)', '{}=effect </(nsubj|csubj|subj|compound)/ ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >/nmod:from/ {}=cause)', '{}=cause >acl ({lemma:/(cause|stimulate|make|derive|trigger|result|lead)/}=trigger >/nmod:(to|in)/ {}=effect)', '{}=cause >acl ({lemma:/give/}=trigger >/nmod:(to|in)/ {}=effect)']
clist = [] #cause list
elist = [] #effect list
tlist = [] #trigger list

dlist = ["A Cochrane review65 suggested that a short course, 3 to 7days, of systemic corticosteroids does not lead to increase in treatment failure or risk of relapse of AECOPD."]

for i, text in enumerate(dlist):
    print(i+1)
    with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner','depparse','coref'], timeout=200000, memory='16G') as client:
        ann = client.annotate(text)
        
        for p, pattern in enumerate(plist):
            matches = client.semgrex(text, pattern)
            if (matches["sentences"][0] == {'length': 0}):
                next
            else:
                clist.append({text + " / pattern: " + str(p+1) : matches["sentences"][0]["0"]["$cause"]["text"]})
                elist.append({text + " / pattern: " + str(p+1) : matches["sentences"][0]["0"]["$effect"]["text"]})
                tlist.append({text + " / pattern: " + str(p+1) : matches["sentences"][0]["0"]["$trigger"]["text"]})
                
# print('---------------')
# print(" ")
# for i in clist:
#     print(i)


1
Starting server with command: java -Xmx16G -cp /Users/kyle/stanford-corenlp-full-2018-10-05//* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 200000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-dea1caf532c0498c.props -preload tokenize,ssplit,pos,lemma,ner,depparse,coref


## Extracted

In [10]:
#Cause
for i in clist:
    print(i)

In [11]:
#Effect
for i in elist:
    print(i)

In [12]:
#Trigger
for i in tlist:
    print(i)